<a href="https://colab.research.google.com/github/nabihafaisal/Natural-Language-Processing/blob/main/B20102130(ASSIGN_3A).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets pandas torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.4 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [ ]:
import nltk
nltk.download('brown')

from nltk.corpus import brown
brown.categories()

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


['adventure',
 'belles_lettres',
 'editorial',
 'fiction',
 'government',
 'hobbies',
 'humor',
 'learned',
 'lore',
 'mystery',
 'news',
 'religion',
 'reviews',
 'romance',
 'science_fiction']

In [ ]:
Xdocwords = []
Y = []


for category in brown.categories():
  for fileid in brown.fileids(category):
    Xdocwords.append(brown.words(fileid))
    Y.append(category)


In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
import re
from transformers import pipeline

# Step 1: Initialize the sentiment analysis pipeline
sentiment_pipeline = pipeline('sentiment-analysis')

# Assuming `Xdocwords[100]` is a list of tokens (words)
sentence = ' '.join([tok.lower() for tok in Xdocwords[450]])

# Step 2: Process the text (split into sentences using line breaks and punctuation)
processed_docs = '\n'.join([s.strip() for s in re.split(r'[,``]', sentence) if s.strip()])

# Split the processed document by line breaks to get each sentence
sentences = processed_docs.split('\n')

# Step 3: Apply sentiment analysis to each sentence
results = sentiment_pipeline(sentences)

# Step 4: Convert labels to binary (POSITIVE -> 1, NEGATIVE -> 0)
binary_results = [(1 if result['label'] == 'POSITIVE' else 0) for result in results]

# for sentence, result in zip(sentences, results):
#     print(f"Sentence: {sentence}\nLabel: {result['label']}\n")

#Step 5: Print each sentence with its corresponding binary sentiment label
df = pd.DataFrame({'sentence': sentences, 'label': binary_results})

# Step 6: Print the DataFrame
print(df)



No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


                                              sentence  label
0                                    francois d'albert      1
1    hungarian-born violinist who made his new york...      1
2    played a return engagement last night in judso...      1
3    a faculty member at depaul university . the ac...      0
4            so that for the first half of the program      0
..                                                 ...    ...
147                                        for example      1
148  a stunning krakowiak that closes the first act...      1
149                                      strasny dwor'      1
150  may be the most beautiful mazurka you are like...      1
151  and the leader of the on-stage band of musicia...      1

[152 rows x 2 columns]


In [ ]:
!pip install datasets transformers


In [ ]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding, EarlyStoppingCallback
from datasets import Dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Assuming `df` contains your data with 'sentence' and 'label' columns
df = pd.DataFrame({'sentence': sentences, 'label': binary_results})
df = df.iloc[:150]

# Train-test split
train_df, test_df = train_test_split(df, test_size=0.1, random_state=42)

# Load tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True)

# Convert data to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Load pre-trained model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=15,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
#    learning_rate=2e-5,
    evaluation_strategy="epoch",  # Use 'eval_strategy' if version >= 4.46
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",  # Ensure that eval_accuracy is used
    greater_is_better=True,
)

# Data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Early stopping callback
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3  # Stop training if there is no improvement for 3 consecutive evaluations
)

# Custom function to compute metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_score(labels, predictions)
    return {"eval_accuracy": accuracy}

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    callbacks=[early_stopping],  # Add early stopping callback
    compute_metrics=compute_metrics,  # Pass the custom metric function
)

# Train the model
trainer.train()

# Evaluate the model
predictions = trainer.predict(test_dataset)
preds = np.argmax(predictions.predictions, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(test_dataset['label'], preds)
print(f"Accuracy: {accuracy:.4f}")

# Classification report
report = classification_report(test_dataset['label'], preds, target_names=["negative", "positive"])
print(report)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.731675,0.333333
2,0.723800,0.714329,0.466667
3,0.713600,0.693087,0.466667
4,0.672300,0.667355,0.600000
5,0.644700,0.644395,0.733333
6,0.599000,0.626553,0.733333
7,0.574800,0.606318,0.733333
8,0.521000,0.583457,0.733333


Accuracy: 0.7333
              precision    recall  f1-score   support

    negative       0.00      0.00      0.00         4
    positive       0.73      1.00      0.85        11

    accuracy                           0.73        15
   macro avg       0.37      0.50      0.42        15
weighted avg       0.54      0.73      0.62        15



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
